In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import xarray as xr
from cordex import cmor as cmor
import cordex as cx

table_dir = "./Tables"

cmor.set_options(table_prefix="CORDEX")


def test_cmorizer_fx():
    ds = cx.cordex_domain("EUR-11", dummy="topo")
    filename = cmor.cmorize_variable(
        ds,
        "orog",
        mapping_table={"orog": {"varname": "topo"}},
        cmor_table=os.path.join(table_dir, "CORDEX-CMIP6_fx.json"),
        dataset_table=os.path.join(table_dir, "CORDEX-CMIP6_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX-CMIP6_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename

In [6]:
f = test_cmorizer_fx()
f

/Users/buntemey/python/packages/py-cordex/cordex/core/domain.py:632: UserWarning: Order of vertices has changed since v0.3.2 to CF Conventions, see https://github.com/euro-cordex/py-cordex/issues/34
  warn(


'CORDEX/CMIP6/CORDEX/EUR-11/GERICS/REMO2020/ECMWF/ECMWF-ERA5/evaluation/r1i1p1f1/fx/orog/gn/v20221020/orog_fx_EUR-11_REMO2020_ECMWF-ERA5_evaluation_r1i1p1f1_gn.nc'

In [4]:
xr.open_dataset(f)

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4)
Coordinates:
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.73 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.05 18.16
    latitude                    (rlat, rlon) float64 ...
    longitude                   (rlat, rlon) float64 ...
Dimensions without coordinates: vertices
Data variables:
    rotated_latitude_longitude  int32 ...
    vertices_latitude           (rlat, rlon, vertices) float64 ...
    vertices_longitude          (rlat, rlon, vertices) float64 ...
    orog                        (rlat, rlon) float32 ...
Attributes: (12/42)
    CORDEX_domain:           EUR-11
    Conventions:             CF-1.7 CMIP-6.2
    activity_id:             CORDEX
    contact:                 gerics-cordex2@hereon.de
    creation_date:           2022-10-20T14:03:55Z
    data_specs_version:      00.00.00
    ...                      ...
    title:                   REMO2020 output prepared for CMIP6
    tracking_id:             hdl:21.14100/01037d19-38e1-4900-bb9f-5a24b39a19aa
    variable_id:             orog
    variant_label:           r1i1p1f1
    license:                 REMO2020 data produced by GERICS is licensed und...
    cmor_version:            3.7.0

In [7]:
!ncdump -h $f

netcdf orog_fx_EUR-11_REMO2020_ECMWF-ERA5_evaluation_r1i1p1f1_gn {
dimensions:
	rlat = 412 ;
	rlon = 424 ;
	bnds = 2 ;
	vertices = 4 ;
variables:
	double rlat(rlat) ;
		rlat:units = "degrees" ;
		rlat:axis = "Y" ;
		rlat:long_name = "latitude in rotated pole grid" ;
		rlat:standard_name = "grid_latitude" ;
	double rlon(rlon) ;
		rlon:units = "degrees" ;
		rlon:axis = "X" ;
		rlon:long_name = "longitude in rotated pole grid" ;
		rlon:standard_name = "grid_longitude" ;
	int rotated_latitude_longitude ;
		rotated_latitude_longitude:grid_mapping_name = "rotated_latitude_longitude" ;
		rotated_latitude_longitude:grid_north_pole_latitude = 39.25 ;
		rotated_latitude_longitude:grid_north_pole_longitude = -162. ;
		rotated_latitude_longitude:north_pole_grid_longitude = 0. ;
	double latitude(rlat, rlon) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:missing_value = 1.e+20 ;
		latitude:_FillValue = 1.e+20 ;
		latitude

In [7]:
from cfchecker.cfchecks import CFChecker

In [8]:
checker = CFChecker()

In [10]:
res = checker.checker(f)

CHECKING NetCDF FILE: CORDEX/CMIP6/CORDEX/GERICS/REMO2020/ECMWF/ECMWF-ERA5/evaluation/r1i1p1f1/fx/orog/gn/v20221011/orog_fx_REMO2020_ECMWF-ERA5_evaluation_r1i1p1f1_gn.nc
Using CF Checker Version 4.1.0
Checking against CF Version CF-1.8
Using Standard Name Table Version 79 (2022-03-19T15:25:54Z)
Using Area Type Table Version 10 (23 June 2020)
Using Standardized Region Name Table Version 4 (18 December 2018)

WARN: (2.6.1): Inconsistency - This netCDF file appears to contain CF-1.7 data, but you've requested a validity check against CF-1.8
WARN: (7.1): Boundary var vertices_latitude should not have attribute units
WARN: (7.1): Boundary var vertices_longitude should not have attribute units

------------------
Checking variable: rlat
------------------

------------------
Checking variable: rlon
------------------

------------------
Checking variable: rotated_latitude_longitude
------------------

------------------
Checking variable: latitude
------------------

------------------
Check

In [19]:
assert not res["global"]["ERROR"]

In [20]:
import json

with open("Tables/CORDEX-CMIP6_CV.json") as f:
    data = json.load(f)

In [24]:
data["CV"].keys()

dict_keys(['required_global_attributes', 'version_metadata', 'activity_id', 'institution_id', 'driving_institution_id', 'source_id', 'driving_source_id', 'source_type', 'frequency', 'grid_label', 'nominal_resolution', 'realm', 'table_id', 'license', 'mip_era', 'experiment_id', 'product', 'tracking_id', 'further_info_url', 'realization_index', 'variant_label', 'data_specs_version', 'Conventions', 'forcing_index', 'initialization_index', 'physics_index'])

In [41]:
for key in data["CV"]:
    filename = "CORDEX-CMIP6_" + key + ".json"
    out = {key: data["CV"][key]}
    with open(filename, "w") as fp:
        json.dump(out, fp, indent=4)
    # print(data["CV"].get(key))

In [28]:
data["CV"]["required_global_attributes"]

['Conventions',
 'CORDEX_domain',
 'activity_id',
 'creation_date',
 'data_specs_version',
 'driving_experiment_id',
 'driving_institution_id',
 'driving_source_id',
 'driving_variant_label',
 'experiment_id',
 'forcing_index',
 'frequency',
 'further_info_url',
 'grid',
 'grid_label',
 'initialization_index',
 'institution',
 'institution_id',
 'license',
 'nominal_resolution',
 'physics_index',
 'product',
 'realization_index',
 'realm',
 'source',
 'source_id',
 'source_type',
 'table_id',
 'tracking_id',
 'variable_id',
 'variant_label']

In [39]:
data["CV"]["activity_id"]

{'CORDEX': 'Coordinated Regional Climate Downscaling Experiment'}

In [42]:
import cmip6_cv

In [43]:
cmip6_cv

<module 'cmip6_cv' from '/Users/buntemey/miniconda3/envs/pyremo-tests/lib/python3.9/site-packages/cmip6_cv/__init__.py'>